A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.


In this project, you will use regression models to predict the number of days a customer rents DVDs for.

As with most data science projects, you will need to pre-process the data provided, in this case, a csv file called rental_info.csv. Specifically, you need to:

Read in the csv file rental_info.csv using pandas. \
Create a column named "rental_length_days" using the columns "return_date" and "rental_date", and add it to the pandas DataFrame. This column should contain information on how many days a DVD has been rented by a customer. \
Create two columns of dummy variables from "special_features", which takes the value of 1 when: \
The value is "Deleted Scenes", storing as a column called "deleted_scenes". \
The value is "Behind the Scenes", storing as a column called "behind_the_scenes".  \
Make a pandas DataFrame called X containing all the appropriate features you can use to run the regression models, avoiding columns that leak data about the target. \
Choose the "rental_length_days" as the target column and save it as a pandas Series called y. \
Following the preprocessing you will need to:

Split the data into X_train, y_train, X_test, and y_test train and test sets, avoiding any features that leak data about the target variable, and include 20% of the total data in the test set. \
Set random_state to 9 whenever you use a function/method involving randomness, for example, when doing a test-train split. \
Recommend a model yielding a mean squared error (MSE) less than 3 on the test set 

Save the model you would recommend as a variable named best_model, and save its MSE on the test set as best_mse.

In [35]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor

df = pd.read_csv('rental_info.csv', parse_dates = ['rental_date', 'return_date'])
df.info()
df.head()

df["rental_length"] = pd.to_datetime(df["return_date"]) - pd.to_datetime(df["rental_date"])
df["rental_length_days"] = (df["return_date"]-df["rental_date"]).dt.days
df.head()

df['deleted_scenes'] = np.where(df['special_features'].str.contains('Deleted Scenes'), 1, 0)
df['behind_the_scenes'] = np.where(df['special_features'].str.contains('Behind the Scenes'), 1, 0)
df.head()

to_drop = ['special_features', 'rental_length', 'rental_length_days', 'rental_date', 'return_date']
X = df.drop(to_drop, axis=1)
y = df.rental_length_days

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

lasso = Lasso(alpha=0.3)
lasso.fit(X_train, y_train)
lasso_coef = lasso.coef_
lasso_coef_dict = dict(zip(X_train.columns, lasso.coef_))
print(lasso_coef_dict)
X_lasso_train, X_lasso_test = X_train.iloc[:, lasso_coef > 0], X_test.iloc[:, lasso_coef > 0]

models = {}

#Test L1 reg
lr_model = LinearRegression()
lr = lr_model.fit(X_lasso_train, y_train)
lr_pred = lr.predict(X_lasso_test)
lr_lasso_mse = MSE(lr_pred, y_test)
models[lr] = lr_lasso_mse
print(f'Lasso reg. MSE: {lr_lasso_mse}')

ols = lr_model.fit(X_train,y_train)
ols_pred = ols.predict(X_test)
ols_mse = MSE(ols_pred, y_test)
print(f'Ordinary LR. MSE: {ols_mse}')

rf = RandomForestRegressor()
param = {'n_estimators': range(1,101), 'max_depth': range(1,11)}
random_search = RandomizedSearchCV(rf, param_distributions = param, cv = 5, n_jobs=-1, random_state = 9)
random_search.fit(X_train, y_train)
best_params = random_search.best_params_
rf = RandomForestRegressor(n_estimators=best_params['n_estimators'], max_depth = best_params['max_depth'], random_state = 9, n_jobs=-1)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
rf_mse = MSE(rf_pred, y_test)
models[rf] = rf_mse
print(f'Random Forest MSE: {rf_mse}')

dec_tree = DecisionTreeRegressor()
param = {'max_features': [None, 'sqrt', 'log2'], 'max_depth': ['None']+[*range(1,11)]}
random_search = RandomizedSearchCV(dec_tree, param_distributions = param, cv = 5, n_jobs=-1, random_state = 9)
random_search.fit(X_train, y_train)
best_params_dc = random_search.best_params_
dec_tree = DecisionTreeRegressor(max_depth = best_params_dc['max_depth'], max_features = best_params_dc['max_features'], random_state=9)
dec_tree.fit(X_train, y_train)
dc_pred = dec_tree.predict(X_test)
dec_tree_MSE = MSE(dc_pred, y_test)
models[dec_tree] = dec_tree_MSE
print(f'Decision Tree MSE: {dec_tree_MSE}')


best_model, best_mse = min(models.items(), key = lambda x: x[1])
print("Best Model:", best_model)
print("Best MSE:", best_mse)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   rental_date       15861 non-null  datetime64[ns, UTC]
 1   return_date       15861 non-null  datetime64[ns, UTC]
 2   amount            15861 non-null  float64            
 3   release_year      15861 non-null  float64            
 4   rental_rate       15861 non-null  float64            
 5   length            15861 non-null  float64            
 6   replacement_cost  15861 non-null  float64            
 7   special_features  15861 non-null  object             
 8   NC-17             15861 non-null  int64              
 9   PG                15861 non-null  int64              
 10  PG-13             15861 non-null  int64              
 11  R                 15861 non-null  int64              
 12  amount_2          15861 non-null  float64            
 13  l